In [ ]:

# This notebook needs pymc and arviz to be installed
# in the used environment. Either install them by the method
# of your choice or uncomment the code below and run it.
#import sys
#!{sys.executable} -m pip install arviz pymc

# Rescoring Using Bayesian and Frequentist Logistic Regression

## Introduction

After the initial scoring of Peptide Spectrum Matches by sage, additional information
such as the deviation of the spectrum's ion mobility or retention time to those predicted for
the peptide can be utilized to refine the scoring. This process is called rescoring.
Here, a statistical model or machine learning approaches are trained to predict wether the peptide
behind a PSM is a target or a decoy. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc as pm
import arviz as az
from typing import List, Tuple, Optional
from numpy.typing import NDArray
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
plt.style.use("bmh")
pd.set_option('display.max_columns', 100)

# The features we want to use for rescoring

features = [
    "score",
    "delta_rt",
    "delta_ims",
    "cosine_similarity",
    "delta_mass",
    "rank",
    "isotope_error",
    "average_ppm",
    "delta_next",
    "delta_best",
    "matched_peaks",
    "longest_b",
    "longest_y",
    "longest_y_pct",
    "missed_cleavages",
    "matched_intensity_pct",
    "poisson",
    "charge",
    "intensity_ms1",
    "intensity_ms2",
    "collision_energy",
    "spectral_entropy_similarity",
    "spectral_correlation_similarity_pearson",
    "spectral_correlation_similarity_spearman",

]

## Preparing the Data

In [ ]:
data_whole = pd.read_csv("./psm_data.csv", usecols=features+["decoy","match_idx"])
# for the sake of runtime of the bayesian model, we will sample 1% of the data
data_shuffle = data_whole.sample(frac=1)
data_downsampled = data_shuffle.sample(frac=0.01)

In [ ]:
match_idx_train = []
match_idx_test = []

size_data = data_downsampled.shape[0]
frac_train = 0
for mi in data_downsampled.match_idx.unique():
    if frac_train < 0.5:
        match_idx_train.append(mi)
        frac_train = data_downsampled.loc[data_downsampled.match_idx.isin(match_idx_train)].shape[0]/size_data
    else:
        match_idx_test.append(mi)

In [ ]:

data_train = data_downsampled[data_downsampled.match_idx.isin(match_idx_train)]
data_test = data_downsampled[data_downsampled.match_idx.isin(match_idx_test)]

y_train = data_train["decoy"].apply(lambda x: 1 if x else 0)
X_train = data_train.loc[:, features]

y_test = data_test["decoy"].apply(lambda x: 1 if x else 0)
X_test = data_test.loc[:, features]
# Standardize the data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
before_rescoring = (data_test.assign(decoy= lambda x: x["decoy"].astype(int), 
            score = lambda x: -x["score"],
            ).sort_values("score")
            .assign(percent_decoy= lambda x: x["decoy"].cumsum() / x["decoy"].sum() * 100)
            .assign(count_target = lambda x: (1-x["decoy"]).cumsum()))

Fig, ax1 = plt.subplots(1,1, figsize=(5,5))
ax2 = ax1.twinx()
cutoff = before_rescoring.where(before_rescoring["percent_decoy"] > 1).dropna().head(1)
sns.histplot(before_rescoring, x="score",ax=ax1,hue="decoy")
sns.lineplot(data=before_rescoring, x="score", y="percent_decoy",ax=ax2, color="red")
ax1.vlines(cutoff["score"], 0, 600, color="black", linestyle="--")
ax1.annotate(f"{cutoff['count_target'].values[0]:.0f} non decoy", (cutoff["score"].values[0]-10, 600), color="black")
plt.show()

## Define the Bayesian Model

In [ ]:
with pm.Model(coords={"peptides": X_train.index.values,"features": X_train.columns}) as logistic_model:
    x_data = pm.Data("x_data", X_train_scaled)
    y_data = pm.Data("y_data", y_train)
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0 , sigma=0.1, shape=X_train_scaled.shape[1], dims=["features"])

    logit_p = pm.Deterministic("logit_p", alpha + pm.math.dot(x_data, beta), dims=["peptides"])
    obs = pm.Bernoulli("obs", logit_p=logit_p, observed=y_data, dims=["peptides"])
        

## Sample from the Posterior Distribution

In [ ]:
with logistic_model as model:
    idata = pm.sample(init="adapt_diag")
    pm.set_data({"x_data": X_test_scaled, "y_data": y_test})
    model.set_dim("peptides", X_test_scaled.shape[0], X_test.index.values)
    model.set_dim("features", X_test_scaled.shape[1], X_test.columns)
    pm.sample_posterior_predictive(idata, var_names=["obs","logit_p"], predictions=True, extend_inferencedata=True)
    

In [ ]:
az.plot_trace(idata, var_names=["alpha", "beta"])
plt.tight_layout()
plt.show()

In [ ]:
feature_importances = idata.posterior.beta.mean(dim=["chain","draw"]).to_dataframe()
sns.barplot(data=feature_importances, x="beta", y="features")
plt.xlabel("Posterior Mean of Logistic Regression Coefficients")
plt.ylabel("Feature")
plt.title("Feature Importances (PyMC)")

In [ ]:
decoy_peptides = data_test.loc[data_test["decoy"],:].index
target_peptides = data_test.loc[~data_test["decoy"],:].index
posterior_decoy = idata.predictions.logit_p.sel(peptides=decoy_peptides).mean(dim=["chain","draw"]).values.flatten()
posterior_target = idata.predictions.logit_p.sel(peptides=target_peptides).mean(dim=["chain","draw"]).values.flatten()

after_rescoring_pymc = (pd.DataFrame({"mean_logit_p": np.concatenate([posterior_decoy, posterior_target]),
                               "decoy": np.concatenate([np.ones_like(posterior_decoy), np.zeros_like(posterior_target)])})
                    .sort_values("mean_logit_p")
                    .assign(percent_decoy=lambda x: x["decoy"].cumsum() / x["decoy"].sum() * 100)
                    .assign(count_target = lambda x: (1-x["decoy"]).cumsum()))
Fig, ax1 = plt.subplots(1,1, figsize=(5,5))
ax2 = ax1.twinx()
cutoff = after_rescoring_pymc.where(after_rescoring_pymc["percent_decoy"] > 1).dropna().head(1)
sns.lineplot(data=after_rescoring_pymc, x="mean_logit_p", y="percent_decoy",ax=ax2, color="red")
sns.histplot(after_rescoring_pymc, x="mean_logit_p",ax=ax1,hue="decoy")
ax1.vlines(cutoff["mean_logit_p"], 0, 660, color="black", linestyle="--")
ax1.annotate(f"{cutoff['count_target'].values[0]:.0f} non decoy", (cutoff["mean_logit_p"].values[0]-5, 660), color="black")
plt.show()

## Frequentist Model

In [ ]:
clf = LogisticRegression(penalty="l2", C=0.01)

clf.fit(X_train_scaled, y_train)
logit_p = clf.decision_function(X_test_scaled)

In [ ]:
after_rescoring_sklearn = (data_test.assign(decoy= lambda x: x["decoy"].astype(int),
                                            logit_p_sklearn = logit_p
            ).sort_values("logit_p_sklearn")
            .assign(percent_decoy= lambda x: x["decoy"].cumsum() / x["decoy"].sum() * 100)
            .assign(count_target = lambda x: (1-x["decoy"]).cumsum()))

Fig, ax1 = plt.subplots(1,1, figsize=(5,5))
ax2 = ax1.twinx()
cutoff = after_rescoring_sklearn.where(after_rescoring_sklearn["percent_decoy"] > 1).dropna().head(1)
sns.histplot(after_rescoring_sklearn, x="logit_p_sklearn",ax=ax1,hue="decoy")
sns.lineplot(data=after_rescoring_sklearn, x="logit_p_sklearn", y="percent_decoy",ax=ax2, color="red")
ax1.vlines(cutoff["logit_p_sklearn"], 0, 600, color="black", linestyle="--")
ax1.annotate(f"{cutoff['count_target'].values[0]:.0f} non decoy", (cutoff["logit_p_sklearn"].values[0]-5, 600), color="black")
plt.show()

In [ ]:
sns.barplot(y=X_test.columns, x=clf.coef_.flatten())
plt.xlabel("Logistic Regression Coefficients")
plt.ylabel("Features")
plt.title("Feature Importance (Sklearn)")
plt.show()